In [1]:
# Project Template Import Cell
%matplotlib inline
import os
import sys
import math

from __future__ import print_function, division
from inspect import getsourcefile

from IPython.display import FileLink

# Standard Notebook Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

CURR_PATH = os.path.abspath(getsourcefile(lambda:0)).rsplit('/', 1)[0] # Get filepath of this notebook
module_path = os.path.join(os.path.dirname(CURR_PATH), 'utils') # Make module path for one dir up and one down into utils
if module_path not in sys.path: # Append to system path list
    sys.path.append(module_path)

import utils as utils ; reload(utils)

print('System : Python ', os.sys.version , '\n\n')

print('Directory Structure:')
print(CURR_PATH)
!tree -cn --filelimit 12

# Keras Setup
import keras
print('\n\nKeras version:' , keras.__version__ ,
      ', backend:' , keras.backend.backend(),
      ', image_format:' , keras.backend.image_data_format())

random_seed = 7

print('\n\nEnvironment : ')
FileLink('../../ana_py27_p2.yml')

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


System : Python  2.7.12 | packaged by conda-forge | (default, Sep  8 2016, 14:22:31) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)] 


Directory Structure:
/home/ubuntu/ana_py27_p2/projects/CervicalCancer
.
├── data
│   ├── additional_Type_1_v2.7z
│   ├── additional_Type_3_v2.7z
│   ├── additional_Type_2_v2.7z
│   ├── train
│   │   ├── Type_1 [250 entries exceeds filelimit, not opening dir]
│   │   ├── Type_2 [781 entries exceeds filelimit, not opening dir]
│   │   └── Type_3 [450 entries exceeds filelimit, not opening dir]
│   ├── test
│   │   └── unknown [512 entries exceeds filelimit, not opening dir]
│   ├── preview
│   ├── results
│   ├── downloads
│   │   ├── sample_submission.csv.zip
│   │   ├── test.7z
│   │   └── train.7z
│   ├── sample
│   │   ├── preview
│   │   ├── results
│   │   ├── train
│   │   │   ├── Type_1 [80 entries exceeds filelimit, not opening dir]
│   │   │   ├── Type_2 [249 entries exceeds filelimit, not opening dir]
│   │   │   └── Type_3 [144 entries exceeds filel

/home/ubuntu/ana_py27_p2/ana_py27_p2.yml

In [2]:
sub = pd.read_csv('data/submissions/sample_submission_000.csv')
sub.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.168805,0.527346,0.303849
1,1.jpg,0.168805,0.527346,0.303849
2,2.jpg,0.168805,0.527346,0.303849
3,3.jpg,0.168805,0.527346,0.303849
4,4.jpg,0.168805,0.527346,0.303849


In [3]:
sub.describe()

,Type_1,Type_2,Type_3
count,5.120000e+02,5.120000e+02,5.120000e+02
mean,1.688049e-01,5.273464e-01,3.038488e-01
std,5.834371e-16,1.555832e-15,1.055743e-15
min,1.688049e-01,5.273464e-01,3.038488e-01
25%,1.688049e-01,5.273464e-01,3.038488e-01
50%,1.688049e-01,5.273464e-01,3.038488e-01
75%,1.688049e-01,5.273464e-01,3.038488e-01
max,1.688049e-01,5.273464e-01,3.038488e-01


In [5]:
print(250/(250+781+450))
print(781/(250+781+450))
print(450/(250+781+450))

0.16880486158
0.527346387576
0.303848750844


In [6]:
# Create a submisssion file using training prevalence rates as probabilities
test_filenames = [f.split('/')[-1] for f in utils.get_non_hidden_dir_contents('data/test/unknown')]

# file number ascending numerical order
test_filenames.sort(key = lambda fn: int(fn.split('.')[0]))
type_1_probs = [250/(250+781+450)]* len(test_filenames)
type_2_probs = [781/(250+781+450)]* len(test_filenames)
type_3_probs = [450/(250+781+450)]* len(test_filenames)

sub_001 = pd.DataFrame(data={'image_name':test_filenames, 'Type_1':type_1_probs, 
                             'Type_2':type_2_probs, 'Type_3':type_3_probs})

cols_ordered = list(sub_001.columns) ; cols_ordered.sort(key = lambda c: c.lower())
sub_001 = sub_001.reindex_axis(cols_ordered, axis=1)

print(sub_001.head(13))
#sub_001.to_csv('data/submissions/sub_001.csv', index=False)

   image_name    Type_1    Type_2    Type_3
0       0.jpg  0.168805  0.527346  0.303849
1       1.jpg  0.168805  0.527346  0.303849
2       2.jpg  0.168805  0.527346  0.303849
3       3.jpg  0.168805  0.527346  0.303849
4       4.jpg  0.168805  0.527346  0.303849
5       5.jpg  0.168805  0.527346  0.303849
6       6.jpg  0.168805  0.527346  0.303849
7       7.jpg  0.168805  0.527346  0.303849
8       8.jpg  0.168805  0.527346  0.303849
9       9.jpg  0.168805  0.527346  0.303849
10     10.jpg  0.168805  0.527346  0.303849
11     11.jpg  0.168805  0.527346  0.303849
12     12.jpg  0.168805  0.527346  0.303849


Use Kaggle cli to submit

`kg submit sub_001.csv -u username -p password -c "intel-mobileodt-cervical-cancer-screening" -m "001"`

This gives a score of 1.00575

In [7]:
# What about submitting using even probabilities?

sub_002 = pd.read_csv('data/submissions/sub_001.csv')
sub_002['Type_1'] = 1/3 ; sub_002['Type_2'] = 1/3  ; sub_002['Type_3'] = 1/3
print(sub_002.head(13))
#sub_002.to_csv('data/submissions/sub_002.csv', index=False)

   image_name    Type_1    Type_2    Type_3
0       0.jpg  0.333333  0.333333  0.333333
1       1.jpg  0.333333  0.333333  0.333333
2       2.jpg  0.333333  0.333333  0.333333
3       3.jpg  0.333333  0.333333  0.333333
4       4.jpg  0.333333  0.333333  0.333333
5       5.jpg  0.333333  0.333333  0.333333
6       6.jpg  0.333333  0.333333  0.333333
7       7.jpg  0.333333  0.333333  0.333333
8       8.jpg  0.333333  0.333333  0.333333
9       9.jpg  0.333333  0.333333  0.333333
10     10.jpg  0.333333  0.333333  0.333333
11     11.jpg  0.333333  0.333333  0.333333
12     12.jpg  0.333333  0.333333  0.333333


Use Kaggle cli to submit

`kg submit sub_002.csv -u username -p password -c "intel-mobileodt-cervical-cancer-screening" -m "002"`

This gives a score of 1.09861

The score gets slightly worse when submitting balanced probabilities

This could mean that the prevalence of cases is approximately proportional to the proportion of training instances for each cancer type

May be able to use this for final review of probabilities, prior to submission